<a href="https://colab.research.google.com/github/wshweta23/Shweta-Waghmare-1/blob/main/SHWETA_WAGHMARE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Table of Contents
1  Libraries
2  Functions
3  Data Prep
4  Terrorism Around the World
5  Countries and Terrorism
6  NLP on Incident Summary
6.1  Regular Expressions
6.1.1  Sites and Hiperlinks
6.1.2  Numbers
6.1.3  Special Characteres
6.1.4  Additional Whitespaces
6.2  Lower Case
6.3  WordCloud
7  References
This kernel is a continuation of the kernel Global Terrorism - Data Understanding, where an extremely important process of understanding the data was carried out in order to go deep into the context and make useful data transformations for further analysis.
This second approach intends to tell a story about Terrorism around the world taking the data provided as a basis. Here, we will apply an exploratory data analysis, look for patterns and explanations related to the context and present the conclusions in a dynamic and visual ways. If you like this kernel, please upvote! This always keep me motivated to do things even better! English is not my mother language... so sorry for any mistake!

We will use libs like Folium, Seaborn, Matplotlib and other usefull tools to try to see:

A big picture of terrorism around the world and its evolution over the years;
Countries with most incidents recorded;
Countries with highest number of victims;
A dashboard for terrorism analysis in some countries;
Incidents that lasted more than 24h (extended = 1);
Major radical groups responsible for terrorist attacks (gname);
Attacks with the highest number of terrorists (nperps);
A WordCloud for attributes like summary corp1, target1 and motive;
Libraries
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import folium
from folium.plugins import FastMarkerCluster, Fullscreen, MiniMap, HeatMap, HeatMapWithTime
import geopandas as gpd
from branca.colormap import LinearColormap
import os
import json
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
from wordcloud import WordCloud, STOPWORDS
Functions
Reading the Data
In previous versions, I separated the analysis into "Data Preparation" and "Storytelling" just to keep in mind that some transformations in data would be necessary to reach the goals. But, as long as the storytelling part took the center stage and grew on large scale, I decided to make the transformations on data only where applicable.

In other words, all the steps required for the analysis will be performed on their respective topic. For now, let's just read the data, filter attributes and go through visualizations!

terr = pd.read_csv('../input/globalterrorismdb_0718dist.csv', encoding='ISO-8859-1')
attribs = ['eventid', 'iyear', 'imonth', 'iday', 'extended', 'country_txt', 'region_txt', 'city', 
                        'latitude', 'longitude', 'specificity', 'summary', 'success', 'suicide', 'attacktype1_txt', 
                        'targtype1_txt', 'copr1', 'target1', 'natlty1_txt', 'gname', 'motive', 'nperps', 
                        'weaptype1_txt', 'nkill', 'nkillter', 'nwound', 'nwoundte', 'ishostkid', 'nhostkid']
terr_data = terr.loc[:, attribs]
terr_data.head()
eventid	iyear	imonth	iday	extended	country_txt	region_txt	city	latitude	longitude	...	gname	motive	nperps	weaptype1_txt	nkill	nkillter	nwound	nwoundte	ishostkid	nhostkid
0	197000000001	1970	7	2	0	Dominican Republic	Central America & Caribbean	Santo Domingo	18.456792	-69.951164	...	MANO-D	NaN	NaN	Unknown	1.0	NaN	0.0	NaN	0.0	NaN
1	197000000002	1970	0	0	0	Mexico	North America	Mexico city	19.371887	-99.086624	...	23rd of September Communist League	NaN	7.0	Unknown	0.0	NaN	0.0	NaN	1.0	1.0
2	197001000001	1970	1	0	0	Philippines	Southeast Asia	Unknown	15.478598	120.599741	...	Unknown	NaN	NaN	Unknown	1.0	NaN	0.0	NaN	0.0	NaN
3	197001000002	1970	1	0	0	Greece	Western Europe	Athens	37.997490	23.762728	...	Unknown	NaN	NaN	Explosives	NaN	NaN	NaN	NaN	0.0	NaN
4	197001000003	1970	1	0	0	Japan	East Asia	Fukouka	33.580412	130.396361	...	Unknown	NaN	NaN	Incendiary	NaN	NaN	NaN	NaN	0.0	NaN
5 rows × 29 columns

Before we get started, we need to ensure the charts will be plotted the way we expect. First, we have to set the name of United States country to same string that we have in the json file used as map (United States of America). Second, let's just change the large string describing the "Vehicle" in weaptype1_txt attribute.

terr_data['country_txt'] = terr_data['country_txt'].apply(lambda x: x.replace('United States', 
                                                                              'United States of America'))
terr_data['weaptype1_txt'] = terr_data['weaptype1_txt'].apply(lambda x: x.split()[0] if 'Vehicle' in x.split() else x)
If you want to see details of a full process of global terrorism data preparation, please go through the Global Terrorism - Data Understanding.

Terrorism Around the World

Well, here we can see clearly that Iraq is the country with the highest number of incidents recorded. The map also shows tooltips with the name of the country, number of incidents and total of victims recorded. Another thing that can be said looking at the map is that the Middle East and South Asia are the regions with the highes number of recorded attacks between 1970 and 2017.


With this Heatmap, we can see places with greater concentration of terrorism incidents from 1970 to 2017. To get more insights, let's see the"evolution" of terrorism year by year.


Now we have a selection bar in the bottom of the map where we can select the terrorist records from a specific year between 1970 and 2017. It is important to cross this information with historical facts, wars and incidents. It's kind of interesting to see how the concentration of incidents starts at North America in the 70s and move to Europe and Middle East region as long as the time goes by.

The most recent data we have is from 2017. Let's plot a global heatmap to see incidents among the months of 2017. Using the selection bar in the bottom, we can see the concentration of terrorism from january to december of 2017.


Countries and Terrorism
Let's change the scenery and see the effects of terrorism in specific countries. First of all, let's take a look at the main countries affected by terrorism.


As we have already seen in our first geographical plot, the highest concentration of incidentes recorded are from Middle East & North Africa. The region represents 27.8% of all records between 1970 and 2017.

In the next plot, we will make a comparison of this historical data with 2017 data, but this time looking at the top 10 countries if highest nuber of terrorist incidents.


With the grap above we can see that Iraq and Afghanistan are the countries with most terrorism occurences in 2017 (and also in all period). Colombia, Peru and El Salvador appear in historica data but don't appear in 2017 data maybe because of past conflicts. Let's make a more specific analysis in some countries to see more details.

Now I present you the dashboard for country-terrorism relationship analysis. We will see details from Iraq, United States, Nigeria, Colombia and Egypt.






NLP on Incident Summary
Well, in this session our goal is to apply some Natural Language Processing techniques to take a look at the words as a tool for understanding the Terrorism around the World. First of all, let's see some elements from the column summary

terr_data['summary'][:10]
0                                                  NaN
1                                                  NaN
2                                                  NaN
3                                                  NaN
4                                                  NaN
5    1/1/1970: Unknown African American assailants ...
6                                                  NaN
7    1/2/1970: Unknown perpetrators detonated explo...
8    1/2/1970: Karl Armstrong, a member of the New ...
9    1/3/1970: Karl Armstrong, a member of the New ...
Name: summary, dtype: object
We can see two points by now:

It is necessary to handle null data on this
Also, we have to eliminate the date information at the beginning of each instance
temp_corpus = terr_data['summary'].dropna()
corpus = temp_corpus.apply(lambda x: x.split(': ')[-1]).values
print(f'We have {len(corpus)} elements on the corpus\n\n')
print(f'Example 1: \n{corpus[1]}\n')
print(f'Example 2: \n{corpus[-1]}')
We have 115562 elements on the corpus


Example 1: 
Unknown perpetrators detonated explosives at the Pacific Gas & Electric Company Edes substation in Oakland, California, United States.  Three transformers were damaged costing an estimated $20,000 to $25,000.  There were no casualties.

Example 2: 
An explosive device was discovered and defused at a plaza in Cotabato City, Maguindanao, Philippines. No group claimed responsibility for the incident.
Regular Expressions
Now that we have already transformed the corpus on an array structure, let's apply some Regular Expressions to deal with non-desired elements. The analysis we will do in the following topics will cover:

Search for Sites and Hiperlinks
Search for Numbers
Search for Special Characteres
Search for Additional Whitespaces
Sites and Hiperlinks
For this task, we will iterate all over the corpus applying the method findall using a specific Regular Expression created for searching sites and hiperlinks. Let's see what we can tell about it.

for c in corpus:
    urls = re.findall('(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', c)
    if len(urls) == 0:
        pass
    else:
        print(f'Description: {list(corpus).index(c)} - Links: {urls}')
Description: 6977 - Links: [('http', 'www.earthliberationfront.com', '/news/2002/020116ml.html')]
Well, it seems that only one attack summary have hiperlinks on it. Let's print it just to confirm.

# Example
corpus[6977]
' ALF cites this incident as an anti-globalization action in their 2001 year-end Direct Action Report, available as of 13 January 2003 at http://www.earthliberationfront.com/news/2002/020116ml.html.'
Ok, our method worked. So we will replace the links with the token "link".

# Replacing sites and hiperlinks
corpus_wo_hiperlinks = []
for c in corpus:
    c = re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', 'link', c)
    corpus_wo_hiperlinks.append(c)
corpus_wo_hiperlinks[6977]
' ALF cites this incident as an anti-globalization action in their 2001 year-end Direct Action Report, available as of 13 January 2003 at link.'
Good! Keep going.

Numbers
As we could see on some examples above, there are incidents descriptions with numbers. Here we will search for those text descriptions and replace the numbers with the token number.

# Example of description with number
corpus_wo_hiperlinks[399]
'At around 8:00 AM, a member of the Black Liberation Army flagged down a police officer driving his radio car in Manhattan, New York, United States.  When the cop pulled over, the perpetrator began stabbing the officer with a knife.  The officer then shot and killed the assailant.  That morning, the New York Times received two anonymous calls stating that a New York policeman would be killed at 8:00 AM unless the BLA received $100,000.'
# Replacing numbers
corpus_wo_numbers = []
for c in corpus_wo_hiperlinks:
    c = re.sub('\d+(?:\.\d*(?:[eE]\d+))?', 'number', c)
    corpus_wo_numbers.append(c)
corpus_wo_numbers[399]
'At around number:number AM, a member of the Black Liberation Army flagged down a police officer driving his radio car in Manhattan, New York, United States.  When the cop pulled over, the perpetrator began stabbing the officer with a knife.  The officer then shot and killed the assailant.  That morning, the New York Times received two anonymous calls stating that a New York policeman would be killed at number:number AM unless the BLA received $number,number.'
We are one step closer to the goal.

Special Characteres
Again, as long as we are looking at incidents descriptions, we faced some special characteres that need to be replaced for further analysis. For this, let's apply another Regular Expresion to replace special characters with whitespace.

# Example with special characteres
corpus_wo_numbers[1113]
'Members of the Evan Mecham Eco-Terrorist International Conspiracy (EMETIC), claimed responsibility for damaging the pylons and cables which supported the main ski chairlift at the Fairfield Snow Bowl Ski Resort in Flagstaff, Arizona in the United States,   There were no casualties, but the property damage was estimated at $number,number and an additional $number,number to heighten security at the resort. In a letter to the ski resort, the group stated that "if trees were not allowed to grow back" and "ski operations not discontinued," then more damage would occur. Marc Leslie Davis, Margaret Katherine Millet, Ilse Washington Asplund, and Marc Andre Baker were arrested and in number, pled guilty and sentenced for their involvement in this incident.'
# Replacing special characteres with whitespace
corpus_text = []
for c in corpus_wo_numbers:
    c = re.sub(r'\W', ' ', c)
    corpus_text.append(c)
corpus_text[1113]
'Members of the Evan Mecham Eco Terrorist International Conspiracy  EMETIC   claimed responsibility for damaging the pylons and cables which supported the main ski chairlift at the Fairfield Snow Bowl Ski Resort in Flagstaff  Arizona in the United States    There were no casualties  but the property damage was estimated at  number number and an additional  number number to heighten security at the resort  In a letter to the ski resort  the group stated that  if trees were not allowed to grow back  and  ski operations not discontinued   then more damage would occur  Marc Leslie Davis  Margaret Katherine Millet  Ilse Washington Asplund  and Marc Andre Baker were arrested and in number  pled guilty and sentenced for their involvement in this incident '
Additional Whitespaces
As we applied Regular Expressions (like this one on session 6.1.3), we generated some additional whitespaces. We can threat it with RegEx as well.

# Removing additional whitespaces
corpus_after_regex = []
for c in corpus_text:
    c = re.sub(r'\s+', ' ', c)
    corpus_after_regex.append(c)
    
corpus_after_regex[1113]
'Members of the Evan Mecham Eco Terrorist International Conspiracy EMETIC claimed responsibility for damaging the pylons and cables which supported the main ski chairlift at the Fairfield Snow Bowl Ski Resort in Flagstaff Arizona in the United States There were no casualties but the property damage was estimated at number number and an additional number number to heighten security at the resort In a letter to the ski resort the group stated that if trees were not allowed to grow back and ski operations not discontinued then more damage would occur Marc Leslie Davis Margaret Katherine Millet Ilse Washington Asplund and Marc Andre Baker were arrested and in number pled guilty and sentenced for their involvement in this incident '
Good! I think we are done here with RegEx

Lower Case
The next step we must on Natural Language Processing is putting all the tokens in lower case. We can do that with the method apply of Pandas DataFrame.

cleaned_corpus = pd.Series(corpus_after_regex).apply(lambda x: x.lower())
cleaned_corpus = list(cleaned_corpus.values)
cleaned_corpus[990]
'the vietnamese organization to exterminate communists and restore the nation claimed responsibility for shooting nguyen van luy and his wife pham thi luu in front of their residence in san francisco california united states van luy a vocal defender of vietnam s communist government was wounded and thi luu was killed '
WordCloud
In this session, we will generate a WordCloud for all descriptions.


I hope you really enjoy this storytelling. Please upvote this kernel to keep me motivated to do even more!

This is not the final version. There is much more to do:

Create a WordCloud for attributes like corp1, target1 and motive;
Look for more Exploratory Data Analysis like:
Incidents that lasted more than 24h (extended = 1);
Major radical groups responsible for terrorist attacks (gname);
Attacks with the highest number of terrorists (nperps);
References
https://nbviewer.jupyter.org/gist/jtbaker/57a37a14b90feeab7c67a687c398142c?flush_cache=true

https://github.com/python-visualization/folium/issues/904

https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf

https://www.kaggle.com/rachan/how-to-folium-for-maps-heatmaps-time-analysis

https://python-visualization.github.io/folium/plugins.html

 